# Bắt đầu

In [2]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import cv2

In [3]:
img = Image.open('./Picture/cat-1.jpg')
img = img.resize((img.size[0],img.size[0]*2 ), Image.LANCZOS)
img_pixel = np.array(img)

In [4]:
height, width, channel = img_pixel.shape

In [5]:
error_id = []
def check(input1, input2):
    global error_id
    error_id = []
    sum = 0
    for i in range(len(input1)):
        for j in range(len(input1[0])):
            if np.abs(input1[i][j]-input2[i][j])!=0:
                error_id.append([i,j])
            sum+=np.abs(input1[i][j]-input2[i][j])
    return sum

In [43]:
def check_avg(input1, input2):
    sum = 0
    count = 0
    for i in range(len(input1)):
        for j in range(len(input1[0])):
            sum+=np.abs(input1[i][j]-input2[i][j])
            count+=1
    return sum/count

# Gray scale

In [6]:
def gray_scale(input, width, height, channel):
    if channel==1:
        return
    return 0.299*input[:,:,0]+0.587*input[:,:,1]+0.114*input[:,:,2]

In [7]:
gray = gray_scale(img_pixel, width, height, channel)
gray_cv = cv2.cvtColor(img_pixel, cv2.COLOR_RGB2GRAY)

In [8]:
gray_round = [[np.rint(e) for e in row] for row in gray]
error_id = []
check(gray_round, gray_cv)

872.0

In [9]:
gray_np = np.dot(img_pixel, [0.299, 0.587, 0.114])

In [10]:
check(gray_np, gray)

0.0

In [11]:
check(np.rint(np.dot(img_pixel, [0.299, 0.587, 0.114])).astype(np.uint8), gray_cv)

C:\Users\lminh\AppData\Local\Temp\ipykernel_1516\2583768796.py:8: RuntimeWarning: overflow encountered in scalar subtract
  if np.abs(input1[i][j]-input2[i][j])!=0:
C:\Users\lminh\AppData\Local\Temp\ipykernel_1516\2583768796.py:10: RuntimeWarning: overflow encountered in scalar subtract
  sum+=np.abs(input1[i][j]-input2[i][j])


222360

# Convolution

In [12]:
sobel_x = np.array([[-1, 0, 1],
                [-2, 0, 2],
                [-1, 0, 1]])
sobel_y = np.array([[-1,-2,-1],
                [ 0, 0, 0],
                [ 1, 2, 1]])

In [13]:
def convolution(input, width, height, filter, filter_size):
    output = []
    for h in range(height):
        row = []
        for w in range(width):
            current_sum = 0
            for f_r in range(filter_size):
                for f_c in range(filter_size):
                    pixel_r = h - int(filter_size/2) + f_r
                    pixel_c = w - int(filter_size/2) + f_c
                    pixel_r = min(max(0, pixel_r), height - 1)
                    pixel_c = min(max(0, pixel_c), width - 1)
                    current_sum += filter[f_r][f_c] * input[pixel_r][pixel_c]
            row.append(current_sum)
        output.append(row)
    return output

In [14]:
np.array(gray_round)

array([[239., 239., 239., ..., 170., 169., 166.],
       [238., 238., 238., ..., 169., 168., 166.],
       [238., 238., 238., ..., 169., 168., 166.],
       ...,
       [135., 129., 125., ..., 181., 180., 176.],
       [133., 125., 119., ..., 181., 180., 176.],
       [130., 122., 116., ..., 181., 180., 176.]])

In [15]:
gradient_x = convolution(gray_round, width, height, sobel_x, 3)
gradient_y = convolution(gray_round, width, height, sobel_y, 3)

In [16]:
gradient_x_cv   = cv2.Sobel(np.array(gray_round).astype(np.uint8), cv2.CV_32F, dx=1, dy=0,borderType=cv2.BORDER_REPLICATE)
gradient_y_cv   = cv2.Sobel(np.array(gray_round).astype(np.uint8), cv2.CV_32F, dx=0, dy=1,borderType=cv2.BORDER_REPLICATE)

In [17]:
check(gradient_x, gradient_x_cv)

0.0

In [18]:
check(gradient_y, gradient_y_cv)

0.0

# Tính hướng và độ lệch

In [19]:
g, theta = cv2.cartToPolar(gradient_x_cv, gradient_y_cv, angleInDegrees=True) 

In [20]:
magnitude = np.sqrt(np.square(gradient_x)+np.square(gradient_y))

In [21]:
check(g, magnitude)

1.838358444177605

In [22]:
direction = np.mod(np.add(360, np.rad2deg(np.arctan2(np.array(gradient_y), np.array(gradient_x)))), 360)

# Tính Histogram của 1 cell

In [57]:
def cell_hist(direction, magnitude, idx, idy, width, height, cell_size, sbin):
    output = [0,0,0,0,0,0,0,0,0]
    for r in range(cell_size):
        for c in range(cell_size):
            cur_r = idy*cell_size + r
            cur_c = idx*cell_size + c
            if cur_r>=height or cur_c >= width:
                break
            quotient = int(direction[cur_r][cur_c]//sbin)
            remainder = direction[cur_r][cur_c] % sbin
            if remainder==0:
                output[quotient] += magnitude[cur_r][cur_c]
            else:
                first_bin = quotient
                second_bin = first_bin+1
                output[first_bin] += magnitude[cur_r][cur_c]*\
                    ((second_bin*sbin - direction[cur_r][cur_c])/(second_bin*sbin - first_bin*sbin))
                second_bin_idx = second_bin
                if second_bin > 8:
                    second_bin_idx = 0
                output[second_bin_idx] += magnitude[cur_r][cur_c]*\
                    ((direction[cur_r][cur_c] - first_bin*sbin)/(second_bin*sbin - first_bin*sbin))
                
    return output

In [58]:
cell_hist(direction, magnitude, 0, 0, width, height, 8, 40)

[0.0, 0, 0, 0, 0, 1.590990257669732, 18.89963051521849, 42.91429757439944, 0]

In [61]:
magnitude[:8,:8]

array([[4.        , 4.        , 4.        , 4.        , 4.        ,
        4.        , 4.        , 4.24264069],
       [4.        , 4.        , 4.        , 4.        , 4.        ,
        4.        , 4.        , 3.16227766],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ]])

In [60]:
np.array(direction[:8,:8])

array([[270.        , 270.        , 270.        , 270.        ,
        270.        , 270.        , 270.        , 225.        ],
       [270.        , 270.        , 270.        , 270.        ,
        270.        , 270.        , 270.        , 251.56505118],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.       

In [24]:
cell_size = (8, 8)
block_size = (2, 2)
nbins = 9
win_size = (img_pixel.shape[1]//cell_size[1]*cell_size[1],
            img_pixel.shape[0] // cell_size[0] * cell_size[0])
block_size_pixel = (block_size[1] * cell_size[1], block_size[0] * cell_size[0])
block_stride = (cell_size[1], cell_size[0])
print('Kích thước bức ảnh crop theo winSize (pixel): ', win_size)
print('Kích thước của 1 block (pixel): ', block_size_pixel)
print('Kích thước của block stride (pixel): ', block_stride)

Kích thước bức ảnh crop theo winSize (pixel):  (1352, 2704)
Kích thước của 1 block (pixel):  (16, 16)
Kích thước của block stride (pixel):  (8, 8)


In [80]:
hog = cv2.HOGDescriptor(_winSize=win_size,
                        _blockSize=block_size_pixel,
                        _blockStride=block_stride,
                        _cellSize=cell_size,
                        _nbins=nbins,
                        _histogramNormType = 0)

In [81]:
n_cells = (img_pixel.shape[0] // cell_size[0], img_pixel.shape[1] // cell_size[1])
print('Kích thước lưới ô vuông (ô vuông): ', n_cells)

Kích thước lưới ô vuông (ô vuông):  (338, 169)


In [83]:
hog_feats = hog.compute(np.array(gray_round).astype(np.uint8))\
               .reshape(n_cells[1] - block_size[1] + 1,
                        n_cells[0] - block_size[0] + 1,
                        block_size[0], block_size[1], nbins) \
               .transpose((1, 0, 2, 3, 4))  

In [85]:
hog_feats.shape

(337, 168, 2, 2, 9)

In [84]:
hog_feats

array([[[[[1.17349282e-01, 4.07119049e-03, 1.21825542e-02, ...,
           4.29411642e-02, 1.43501842e-02, 1.17349282e-01],
          [8.80170017e-02, 8.95661954e-03, 2.68016197e-02, ...,
           1.71225920e-01, 5.72207645e-02, 8.80170017e-02]],

         [[3.50637972e-01, 8.95661954e-03, 2.68016197e-02, ...,
           4.23368253e-02, 1.41482251e-02, 3.50637972e-01],
          [2.17750281e-01, 1.97045598e-02, 5.89635670e-02, ...,
           1.75628915e-01, 5.86921722e-02, 2.17750281e-01]]],


        [[[1.68131888e-01, 1.04303593e-02, 3.12115178e-02, ...,
           8.66310596e-02, 2.89506502e-02, 1.68131888e-01],
          [2.07018241e-01, 1.52268019e-02, 4.55642752e-02, ...,
           1.40794650e-01, 4.70511764e-02, 2.07018241e-01]],

         [[8.49203020e-02, 1.11094108e-02, 3.32434699e-02, ...,
           3.84777337e-01, 1.28586143e-01, 8.49203020e-02],
          [1.56825185e-01, 1.77556444e-02, 5.31315021e-02, ...,
           3.43417108e-01, 1.14764214e-01, 1.56825185e-01]]]

In [51]:
from skimage import feature
H = feature.hog(gray_round, orientations=9, pixels_per_cell=(8, 8),
                cells_per_block=(2, 2), transform_sqrt=True, block_norm="L2")

print('Kích thước hog features: ', H.shape)

Kích thước hog features:  (2038176,)


In [52]:
H

array([0.02087984, 0.        , 0.        , ..., 0.        , 0.        ,
       0.0342105 ])

# Chuẩn hóa vector histogram trong 1 block
Trong 1 block gồm 2x2 ô
--> Kích thước vector histogram = 1x36 (4 cái 1x9)

1. L2-norm: $$f = \frac{v}{\sqrt{||v_2^2||+e^2}}$$

2. L1-norm:

3. L1-sqrt

# Chuẩn hóa vector histogram trong toàn bộ ảnh
Thao tác dịch giống conv


# --> Nối toàn bộ vector lại sẽ được 1 vector đặc trưng cuối cùng. Thuật toán kết thúc

# Code phần chạy thuật toán